In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
data = pd.read_csv('Clean_Dataset.csv', index_col=0)

In [42]:
data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 27.5+ MB


In [44]:
data.describe()

,duration,days_left,price
count,300153.000000,300153.000000,300153.000000
mean,12.221021,26.004751,20889.660523
std,7.191997,13.561004,22697.767366
min,0.830000,1.000000,1105.000000
25%,6.830000,15.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.170000,38.000000,42521.000000
max,49.830000,49.000000,123071.000000


In [45]:
# remove rows with business class
economy = data[data['class'] != 'Business'].copy(deep=True)
economy['class'].unique()

array(['Economy'], dtype=object)

In [46]:
economy.shape

(206666, 11)

In [47]:
def map_stops(x):
    if x == 'zero':
        return 0
    elif x == 'one':
        return 1
    else:
        return 2

In [48]:
economy['stops'] = economy['stops'].apply(map_stops)

In [49]:
# remove rows with 2 or more stops
zero_or_one_stop = economy[economy['stops'] <= 1].copy(deep=True)

In [50]:
zero_or_one_stop['stops'].unique()

array([0, 1])

In [51]:
zero_or_one_stop.shape

(194463, 11)

In [52]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] > lower_bound) & (df[column] < upper_bound)].copy(deep=True)

In [53]:
no_outliers = remove_outliers(zero_or_one_stop, 'price')

In [54]:
no_outliers.shape

(180604, 11)

In [55]:
no_outliers.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,0,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,0,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,0,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,0,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,0,Morning,Mumbai,Economy,2.33,1,5955


In [56]:
trimmed = no_outliers.drop(columns=["airline", "flight", "class"]).copy(deep=True)

In [57]:
trimmed.head()

,source_city,departure_time,stops,arrival_time,destination_city,duration,days_left,price
0,Delhi,Evening,0,Night,Mumbai,2.17,1,5953
1,Delhi,Early_Morning,0,Morning,Mumbai,2.33,1,5953
2,Delhi,Early_Morning,0,Early_Morning,Mumbai,2.17,1,5956
3,Delhi,Morning,0,Afternoon,Mumbai,2.25,1,5955
4,Delhi,Morning,0,Morning,Mumbai,2.33,1,5955


In [58]:
dummies = pd.get_dummies(trimmed)
dummies.head()

,stops,duration,days_left,price,source_city_Bangalore,source_city_Chennai,source_city_Delhi,source_city_Hyderabad,source_city_Kolkata,source_city_Mumbai,...,arrival_time_Evening,arrival_time_Late_Night,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai
0,0,2.17,1,5953,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
1,0,2.33,1,5953,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,0,2.17,1,5956,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,0,2.25,1,5955,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,0,2.33,1,5955,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [59]:
X = dummies.drop(columns="price")
y = dummies["price"]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test = scaler.transform(y_test.values.reshape(-1, 1))

In [62]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [63]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [64]:
y_pred = dt.predict(X_test)

In [65]:
mean_squared_error(y_test, y_pred)

np.float64(0.24666343061433343)

In [66]:
rf = RandomForestRegressor()

In [70]:
y_train = y_train.ravel()
y_test = y_test.ravel()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [71]:
y_pred = rf.predict(X_test)

In [72]:
mean_squared_error(y_test, y_pred)

np.float64(0.16021336956584792)